In [1]:
from random import seed
from random import randrange
from csv import reader
from math import *

# Load a CSV file
def csv_import(filename):
	file_open = open(filename)
	l = reader(file_open)
	df = list(l)
	return df


# get_split a dataframe into k folds
def k_fold_get_split(dataframe, k_folds):
	dataframe_get_split = list()
	dataframe_copy = list(dataframe)
	fold_size = int(len(dataframe) / k_folds)
	for i in range(k_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataframe_copy))
			fold.append(dataframe_copy.pop(index))
		dataframe_get_split.append(fold)
	return dataframe_get_split

# Calculate accuracy of the model
def accuracy_score(actual, predicted):
	true = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			true += 1
	return true / float(len(actual)) * 100.0

# Evaluate decision tree using a cross validation get_split
def dt_implement(dataframe, algorithm, k_folds, *args):
	folds = k_fold_get_split(dataframe, k_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for rows in fold:
			rows_copy = list(rows)
			test_set.append(rows_copy)
			rows_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [rows[-1] for rows in fold]
		accuracy = accuracy_score(actual, predicted)
		scores.append(accuracy)
	return scores

# split a dataframe based on feature and feature value
def get_split_node(index, value, dataframe):
	left_side, right_side = list(), list()
	for rows in dataframe:
		if rows[index] < value:
			left_side.append(rows)
		else:
			right_side.append(rows)
	return left_side, right_side

# Calculate the entropy index for a get_split dataframe
def entropy_index(groups, classes):
    total_instances = float(sum([len(group) for group in groups]))
    #print("total number of instances", total_instances)
    
    entropy = 0.0
    for group in groups:
        size=float(len(group))
        #print("size of groups is", group, size)
        #avoid divide by zero
        if size==0:
            continue
        score=0
        # score the group based on the score for each class
        for class_val in classes:
            p = ([row[-1] for row in group].count(class_val)) / size 
            #print(p)
            if p <=0:
                score=0
            else:
                score = -1*(p *log(p,2))
            score = score+score
            #print("score",score)
        entropy += score * (size / total_instances)
    return entropy

# Select the best split for a dataframe
def best_get_split(dataframe):
	class_values = list(set(rows[-1] for rows in dataframe))
	tree_index, tree_value, tree_score, tree_groups = 9999, 9999, 9999, None
	for index in range(len(dataframe[0])-1):
		for rows in dataframe:
			groups = get_split_node(index, rows[index], dataframe)
			entropy = entropy_index(groups, class_values)
			if entropy < tree_score:
				tree_index, tree_value, tree_score, tree_groups = index, rows[index], entropy, groups
	return {'index':tree_index, 'value':tree_value, 'groups':tree_groups}

# Create a terminal node value
def terminal_node(group):
	output = [rows[-1] for rows in group]
	return max(set(output), key=output.count)

# Create child get_splits for a node or make terminal
def get_split(node, max_depth, min_size, depth):
	left_side, right_side = node['groups']
	del(node['groups'])
	# check for a no split
	if not left_side or not right_side:
		node['left_side'] = node['right_side'] = terminal_node(left_side + right_side)
		return
	# check for maximum depth
	if depth >= max_depth:
		node['left_side'], node['right_side'] = terminal_node(left_side), terminal_node(right_side)
		return
	# process left_side child
	if len(left_side) <= min_size:
		node['left_side'] = terminal_node(left_side)
	else:
		node['left_side'] = best_get_split(left_side)
		get_split(node['left_side'], max_depth, min_size, depth+1)
	# process right_side child
	if len(right_side) <= min_size:
		node['right_side'] = terminal_node(right_side)
	else:
		node['right_side'] = best_get_split(right_side)
		get_split(node['right_side'], max_depth, min_size, depth+1)

# Build a decision tree
def tree_building(train, max_depth, min_size):
	root = best_get_split(train)
	get_split(root, max_depth, min_size, 1)
	return root

# Do prediction with a decision tree
def predict_dt(node, rows):
	if rows[node['index']] < node['value']:
		if isinstance(node['left_side'], dict):
			return predict_dt(node['left_side'], rows)
		else:
			return node['left_side']
	else:
		if isinstance(node['right_side'], dict):
			return predict_dt(node['right_side'], rows)
		else:
			return node['right_side']

# Tree Algorithm
def dt(train, test, max_depth, min_size):
	tree = tree_building(train, max_depth, min_size)
	prediction = list()
	for rows in test:
		predict_dtion = predict_dt(tree, rows)
		prediction.append(predict_dtion)
	return(prediction)




In [ ]:
# Decision tree on wine dataset with 10 fold cross validation (entropy)
seed(3250)
# load and prepare data
filename = 'wine-dataset.csv'
dataframe = csv_import(filename)
k_folds = 10
max_depth = 5
min_size = 10
scores = dt_implement(dataframe, dt, k_folds, max_depth, min_size)
print('Scores using Entropy: %.4f%%' % scores)
print('Mean Accuracy using Entropy: %.4f%%' % (sum(scores)/float(len(scores))))

In [ ]:
# Calculate the gini index for a get_split dataframe
def gini_index(groups, classes):
    total_instances = float(sum([len(group) for group in groups]))
    #print("total number of instances", total_instances)
    
    gini = 0.0
    for group in groups:
        size=float(len(group))
        #print("size of groups is", group, size)
        #avoid divide by zero
        if size==0:
            continue
        score=0
        # score the group based on the score for each class
        for class_val in classes:
            p = ([row[-1] for row in group].count(class_val)) / size 

            score = p*p
            #print("score",score)
        gini += (1-score) * (size / total_instances)
    return gini

# Select the best get_split for a dataframe
def best_get_split_gini(dataframe):
	class_values = list(set(rows[-1] for rows in dataframe))
	tree_index, tree_value, tree_score, tree_groups = 9999, 9999, 9999, None
	for index in range(len(dataframe[0])-1):
		for rows in dataframe:
			groups = get_split_node(index, rows[index], dataframe)
			gini = gini_index(groups, class_values)
			if gini < tree_score:
				tree_index, tree_value, tree_score, tree_groups = index, rows[index], gini, groups
	return {'index':tree_index, 'value':tree_value, 'groups':tree_groups}

# Build a decision tree
def tree_building_gini(train, max_depth, min_size):
	root = best_get_split_gini(train)
	get_split(root, max_depth, min_size, 1)
	return root

# Evaluate decision tree using a cross validation get_split
def dt_implement(dataframe, algorithm, k_folds, *args):
	folds = k_fold_get_split(dataframe, k_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for rows in fold:
			rows_copy = list(rows)
			test_set.append(rows_copy)
			rows_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [rows[-1] for rows in fold]
		accuracy = accuracy_score(actual, predicted)
		scores.append(accuracy)
	return scores

# Tree Algorithm
def dt_gini(train, test, max_depth, min_size):
	tree = tree_building_gini(train, max_depth, min_size)
	prediction = list()
	for rows in test:
		predict_dtion = predict_dt(tree, rows)
		prediction.append(predict_dtion)
	return(prediction)

In [ ]:
# Decision tree on wine dataset with 10 fold cross validation (gini)
seed(3250)
# load and prepare data
filename = 'wine-dataset.csv'
dataframe = csv_import(filename)
k_folds = 10
max_depth = 5
min_size = 10
scores = dt_implement(dataframe, dt_gini, k_folds, max_depth, min_size)
print('Scores using Gini: %.4f%%' % scores)
print('Mean Accuracy using Gini: %.4f%%' % (sum(scores)/float(len(scores))))